In [19]:
from microphone import record_audio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import librosa
import matplotlib.mlab as mlab
from scipy.ndimage.morphology import generate_binary_structure
from scipy.ndimage.morphology import iterate_structure
from typing import Tuple, List
from numba import njit

In [20]:
import pickle

In [21]:
def micRecord(time=10):
    frames, rate = record_audio(time)
    return np.hstack([np.frombuffer(i, np.int16) for i in frames]), rate

In [22]:
def getFile(path):
    recorded_audio, sampling_rate = librosa.load(path, 
                                                 sr=44100, 
                                                 mono=True, 
                                                 offset=1.5, 
                                                 duration=20)
    return recorded_audio, sampling_rate

In [23]:
def userinput():
    while True:
        audioType = input("u for Upload, r for Record: ")
        if audioType == 'u':
            path = input("Enter path to file: ")
            samples, rate = getFile(path)
            break
        elif audioType == 'r':
            samples, rate = micRecord()
            break
        print("Invalid input. Try again.")
    # print(rate)
    
    return samples, rate

In [24]:
def ecdf(data):
    """Returns (x) the sorted data and (y) the empirical cumulative-proportion
    of each datum.
    
    Parameters
    ----------
    data : numpy.ndarray, size-N
    
    Returns
    -------
    Tuple[numpy.ndarray shape-(N,), numpy.ndarray shape-(N,)]
        Sorted data, empirical CDF values"""
    data = np.asarray(data).ravel()  # flattens the data
    y = np.linspace(1 / len(data), 1, len(data))  # stores the cumulative proportion associated with each sorted datum
    x = np.sort(data)
    return x, y

In [25]:
@njit
def _peaks(
    data_2d: np.ndarray, nbrhd_row_offsets: np.ndarray, nbrhd_col_offsets: np.ndarray, amp_min: float
) -> List[Tuple[int, int]]:
    """
    A Numba-optimized 2-D peak-finding algorithm.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected.
    nbrhd_row_offsets : numpy.ndarray, shape-(N,)
        The row-index offsets used to traverse the local neighborhood.
        
        E.g., given the row/col-offsets (dr, dc), the element at 
        index (r+dr, c+dc) will reside in the neighborhood centered at (r, c).
    
    nbrhd_col_offsets : numpy.ndarray, shape-(N,)
        The col-index offsets used to traverse the local neighborhood. See
        `nbrhd_row_offsets` for more details.
        
    amp_min : float
        All amplitudes equal to or below this value are excluded from being
        local peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned in 
        column-major order
    """
    peaks = []  # stores the (row, col) locations of all the local peaks

    # Iterating over each element in the the 2-D data 
    # in column-major ordering
    #
    # We want to see if there is a local peak located at
    # row=r, col=c
    for c, r in np.ndindex(*data_2d.shape[::-1]):
        if data_2d[r, c] <= amp_min:
            # The amplitude falls beneath the minimum threshold
            # thus this can't be a peak.
            continue
        
        # Iterating over the neighborhood centered on (r, c) to see
        # if (r, c) is associated with the largest value in that
        # neighborhood.
        #
        # dr: offset from r to visit neighbor
        # dc: offset from c to visit neighbor
        for dr, dc in zip(nbrhd_row_offsets, nbrhd_col_offsets):
            if dr == 0 and dc == 0:
                # This would compare (r, c) with itself.. skip!
                continue

            if not (0 <= r + dr < data_2d.shape[0]):
                # neighbor falls outside of boundary.. skip!
                continue

            if not (0 <= c + dc < data_2d.shape[1]):
                # neighbor falls outside of boundary.. skip!
                continue

            if data_2d[r, c] < data_2d[r + dr, c + dc]:
                # One of the amplitudes within the neighborhood
                # is larger, thus data_2d[r, c] cannot be a peak
                break
        else:
            # if we did not break from the for-loop then (r, c) is a local peak
            peaks.append((r, c))
    return peaks

In [26]:
def local_peak_locations(data_2d: np.ndarray, neighborhood: np.ndarray, amp_min: float):
    """
    Defines a local neighborhood and finds the local peaks
    in the spectrogram, which must be larger than the specified `amp_min`.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected
    
    neighborhood : numpy.ndarray, shape-(h, w)
        A boolean mask indicating the "neighborhood" in which each
        datum will be assessed to determine whether or not it is
        a local peak. h and w must be odd-valued numbers
        
    amp_min : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned
        in column-major ordering.
    
    Notes
    -----
    The local peaks are returned in column-major order, meaning that we 
    iterate over all nbrhd_row_offsets in a given column of `data_2d` in search for
    local peaks, and then move to the next column.
    """

    # We always want our neighborhood to have an odd number
    # of nbrhd_row_offsets and nbrhd_col_offsets so that it has a distinct center element
    assert neighborhood.shape[0] % 2 == 1
    assert neighborhood.shape[1] % 2 == 1
    
    # Find the indices of the 2D neighborhood where the 
    # values were `True`
    #
    # E.g. (row[i], col[i]) stores the row-col index for
    # the ith True value in the neighborhood (going in row-major order)
    nbrhd_row_indices, nbrhd_col_indices = np.where(neighborhood)
    

    # Shift the neighbor indices so that the center element resides 
    # at coordinate (0, 0) and that the center's neighbors are represented
    # by "offsets" from this center element.
    #
    # E.g., the neighbor above the center will has the offset (-1, 0), and 
    # the neighbor to the right of the center will have the offset (0, 1).
    nbrhd_row_offsets = nbrhd_row_indices - neighborhood.shape[0] // 2
    nbrhd_col_offsets = nbrhd_col_indices - neighborhood.shape[1] // 2

    return _peaks(data_2d, nbrhd_row_offsets, nbrhd_col_offsets, amp_min=amp_min)

In [27]:
def find_min_amp(spectrogram, amp_threshold):
    log_S = np.log(spectrogram).ravel()  # flattened array
    ind = round(len(log_S) * amp_threshold)
    cutoff_log_amplitude = np.partition(log_S, ind)[ind]
    return cutoff_log_amplitude

In [28]:
def peak_extract(samples, sampling_rate, *, amp_threshold=0.75, neighborhood_rank=2, neighborhood_connectivity=1, neighborhood_iterations=20):
	
    """
    Extracts peaks from a spectrogram created from the sample data.
    
    Parameters
    ----------
    samples : numpy.ndarray
        Array of audio samples
	
	sampling_rate : int
		The sampling rate of the audio samples
        
    amp_threshold : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
	neighborhood_rank : int
	neighborhood_connectivity : int
	neighborhood_iterations : int
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned
        in column-major ordering.
    """
    
    time = np.arange(len(samples)) / sampling_rate

    base_structure = generate_binary_structure(neighborhood_rank,neighborhood_connectivity)
    neighborhood = iterate_structure(base_structure, neighborhood_iterations)

    spectrogram, freqs, times = mlab.specgram(
		samples,
		NFFT=4096,
		Fs=sampling_rate,
		window=mlab.window_hanning,
		noverlap=int(4096 / 2)
	)

    spectrogram = np.clip(spectrogram, 10**-20, None)
    
    amp_min = find_min_amp(spectrogram, amp_threshold)

    return local_peak_locations(spectrogram, neighborhood, amp_min)

In [52]:
samples, sample_rate = userinput()
peaks = peak_extract(samples, sample_rate)

u for Upload, r for Record: u
Enter path to file: C:\Users\ejian\CogWorks-2022-Gausslien-Audio-Capstone\Songs2\Piano_Note_C_Sound_Effect_(getmp3.pro).mp3


C:\Users\ejian\anaconda3\envs\week1\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [53]:
peaks

[(1, 0),
 (24, 0),
 (49, 0),
 (73, 0),
 (97, 0),
 (122, 0),
 (171, 0),
 (222, 0),
 (273, 0),
 (324, 0),
 (351, 0),
 (405, 0),
 (478, 0),
 (522, 0),
 (726, 0),
 (765, 0),
 (1240, 0),
 (2026, 0),
 (2027, 0),
 (2028, 0),
 (2029, 0),
 (2030, 0),
 (2031, 0),
 (2032, 0),
 (2033, 0),
 (2034, 0),
 (2035, 0),
 (2036, 0),
 (2037, 0),
 (2038, 0),
 (2039, 0),
 (2040, 0),
 (2041, 0),
 (2042, 0),
 (2043, 0),
 (2044, 0),
 (2045, 0),
 (2046, 0),
 (2047, 0),
 (2048, 0),
 (147, 1),
 (1552, 1),
 (2027, 1),
 (2028, 1),
 (2029, 1),
 (2030, 1),
 (2031, 1),
 (2032, 1),
 (2033, 1),
 (2034, 1),
 (2035, 1),
 (2036, 1),
 (2037, 1),
 (2038, 1),
 (2039, 1),
 (2040, 1),
 (2041, 1),
 (2042, 1),
 (2043, 1),
 (2044, 1),
 (2045, 1),
 (2046, 1),
 (2047, 1),
 (2048, 1),
 (196, 2),
 (247, 2),
 (298, 2),
 (935, 2),
 (1477, 2),
 (1817, 2),
 (2028, 2),
 (2029, 2),
 (2030, 2),
 (2031, 2),
 (2032, 2),
 (2033, 2),
 (2034, 2),
 (2035, 2),
 (2036, 2),
 (2037, 2),
 (2038, 2),
 (2039, 2),
 (2040, 2),
 (2041, 2),
 (2042, 2),
 (2043,

In [54]:
"""
Loads and returns the dictionary of fingerprints
"""
def loadPeaks():
    with open("peaks.pkl", mode="rb") as opened_file:
        return pickle.load(opened_file)

"""
Saves the passed dictionary of song samples
The database can be switched if a different dictionary is passed
"""
def savePeaks(peak_dict: dict):
    with open("peaks.pkl", mode="wb") as opened_file:
        pickle.dump(peak_dict, opened_file)


In [55]:
peak_dict = {}
for i,v in enumerate(peaks):
    peak_dict.setdefault(peaks[i][1], []).append(peaks[i][0])
peak_dict

{0: [1,
  24,
  49,
  73,
  97,
  122,
  171,
  222,
  273,
  324,
  351,
  405,
  478,
  522,
  726,
  765,
  1240,
  2026,
  2027,
  2028,
  2029,
  2030,
  2031,
  2032,
  2033,
  2034,
  2035,
  2036,
  2037,
  2038,
  2039,
  2040,
  2041,
  2042,
  2043,
  2044,
  2045,
  2046,
  2047,
  2048],
 1: [147,
  1552,
  2027,
  2028,
  2029,
  2030,
  2031,
  2032,
  2033,
  2034,
  2035,
  2036,
  2037,
  2038,
  2039,
  2040,
  2041,
  2042,
  2043,
  2044,
  2045,
  2046,
  2047,
  2048],
 2: [196,
  247,
  298,
  935,
  1477,
  1817,
  2028,
  2029,
  2030,
  2031,
  2032,
  2033,
  2034,
  2035,
  2036,
  2037,
  2038,
  2039,
  2040,
  2041,
  2042,
  2043,
  2044,
  2045,
  2046,
  2047,
  2048],
 3: [863,
  1273,
  1392,
  1529,
  2029,
  2030,
  2031,
  2032,
  2033,
  2034,
  2035,
  2036,
  2037,
  2038,
  2039,
  2040,
  2041,
  2042,
  2043,
  2044,
  2045,
  2046,
  2047,
  2048],
 4: [891,
  1007,
  1326,
  1735,
  2028,
  2029,
  2030,
  2031,
  2032,
  2033,
  2034,
  

In [56]:
for i in range(sorted(peak_dict.keys())[-1]):
    if i not in peak_dict.keys():
        peak_dict[i] = []
        
        

In [57]:
peak_dict = dict(sorted(peak_dict.items()))

In [58]:
for i in peak_dict.values():
    for f in i:
        if f == 0:
            i.remove(0)
            

In [50]:
peak_dict

{0: [675, 802, 1580, 1706],
 1: [397, 843, 872, 1149],
 2: [1303, 1348],
 3: [1107, 2013],
 4: [648, 1368, 1688],
 5: [1010],
 6: [285, 414, 467, 1266, 1418, 1669, 1806],
 7: [564, 940, 1769],
 8: [503],
 9: [202, 314, 606, 1509],
 10: [1247, 1971],
 11: [],
 12: [],
 13: [3],
 14: [773, 1205],
 15: [],
 16: [341, 1163],
 17: [],
 18: [1357, 1458, 1638, 1731, 1902],
 19: [885, 1024, 1275],
 20: [1557, 1820],
 21: [],
 22: [],
 23: [1434, 1678],
 24: [275, 732, 787],
 25: [999, 1574],
 26: [369, 536],
 27: [438, 760],
 28: [],
 29: [814],
 30: [606],
 31: [564, 1079, 1374],
 32: [202],
 33: [843, 982, 1126, 1395, 1602],
 34: [94, 1491],
 35: [],
 36: [],
 37: [348, 398, 773, 2041],
 38: [1693],
 39: [1637],
 40: [966, 1163, 1513, 1819],
 41: [],
 42: [],
 43: [],
 44: [261, 940, 1653],
 45: [],
 46: [467, 1709, 1754, 2013],
 47: [648, 1482],
 48: [1247],
 49: [784, 1010],
 50: [],
 51: [313, 1107, 1284, 1617],
 52: [1546, 1775],
 53: [355, 1795, 1834],
 54: [],
 55: [815],
 56: [49],
 5

In [59]:
savePeaks(peak_dict)